In [1]:
from __future__ import absolute_import, print_function
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights, resnet34
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

c:\Users\Noel\anaconda3\envs\torchCUDA\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


In [2]:
weights = None  # ResNet18_Weights.DEFAULT
model = resnet34(weights=weights)

# reshape last layer.
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)
model.load_state_dict(torch.load(r"C:\Users\Noel\Documents\THESIS\Feature Visualization\Weights\resnet34_torchvision\test72_epoch446.pth"))
# Set model to evaluation mode and send to device
model.to(device).eval()

layers_of_interest = [name for name, _ in model.named_modules() if "conv" in name or "fc" in name]

print(layers_of_interest)


['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer1.2.conv1', 'layer1.2.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer2.2.conv1', 'layer2.2.conv2', 'layer2.3.conv1', 'layer2.3.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer3.2.conv1', 'layer3.2.conv2', 'layer3.3.conv1', 'layer3.3.conv2', 'layer3.4.conv1', 'layer3.4.conv2', 'layer3.5.conv1', 'layer3.5.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2', 'layer4.2.conv1', 'layer4.2.conv2', 'fc']


This cell works, but is technically wrong and would not 'fly' if the notebook was to be converted to an actual python script.

In [3]:
# layer_activations = {}

# def hook_wrapper(name: str):
#     def hook_fn(module: nn.Module, input: torch.Tensor, output: torch.Tensor) -> None:
#         layer_activations[name] = output
#     return hook_fn


# for name, layer in model.named_modules():
#     if name in layers_of_interest:
#         layer.register_forward_hook(hook_wrapper(name))

To combat this, I made the hooks into objects of a hook class, holding both the output and the hook function, thus creating dictionary entries 
of a key/value pair of name/Hook_Layer object.

In [3]:
class Hook_Layer():
    def __init__(self, layer) -> None:
        self.hook = layer.register_forward_hook(self.hook_fn)
        self.output = None

    def hook_fn(self, layer, input, output):
        self.output = output
    
    def __call__(self):
        return self.output

layer_activations = {}
for name, layer in model.named_modules():
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        layer_activations[name] = Hook_Layer(layer)

print(layer_activations.values())

dict_values([<__main__.Hook_Layer object at 0x000002DB35D05120>, <__main__.Hook_Layer object at 0x000002DB35D06EF0>, <__main__.Hook_Layer object at 0x000002DB35D064A0>, <__main__.Hook_Layer object at 0x000002DB35D06320>, <__main__.Hook_Layer object at 0x000002DB35D06230>, <__main__.Hook_Layer object at 0x000002DB35D060E0>, <__main__.Hook_Layer object at 0x000002DB35D05F60>, <__main__.Hook_Layer object at 0x000002DB35D05690>, <__main__.Hook_Layer object at 0x000002DB35D05630>, <__main__.Hook_Layer object at 0x000002DB35D06590>, <__main__.Hook_Layer object at 0x000002DB35D066B0>, <__main__.Hook_Layer object at 0x000002DB35D06830>, <__main__.Hook_Layer object at 0x000002DB35D06950>, <__main__.Hook_Layer object at 0x000002DB35D06A70>, <__main__.Hook_Layer object at 0x000002DB35D06BF0>, <__main__.Hook_Layer object at 0x000002DB35D06D10>, <__main__.Hook_Layer object at 0x000002DB35D07A30>, <__main__.Hook_Layer object at 0x000002DB35D07C70>, <__main__.Hook_Layer object at 0x000002DB35D07D30>,

In [4]:
# Create a dataset class that extends ImageFolder while
# simultaneously returning a 3 way Tuple, instead of the
# original that contains 2 elements.
# For that reason we must define a new __getitem__ method.
class ImageFolderWithPaths(ImageFolder):
    """Dataset class extending ImageFolder dataset,
        returning Tuple.
        
        Returns:
                Tuple[img[torch.Tensor],
                      label[int],
                      path[str]]
        """
    def __getitem__(self, index: int):
        # Super the __getitem__ of base class
        img, label = super().__getitem__(index)
        # Extract the path of each image in the dataset
        path = self.imgs[index][0]
        # Return new tuple with 3 elements
        return (img, label, path)

    

In [5]:
batch_size = 64


transforms = T.Compose([T.Resize(224),
                        # T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize([0.5162, 0.4644, 0.3975],
                                    [0.2724, 0.2640, 0.2574])
                        ])

dataset = ImageFolderWithPaths(root=r"C:\Users\Noel\Documents\THESIS\Outputs_Feature_Visualization\test72outputs",
                               transform=transforms)

dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, num_workers=0)
print("Dataloader Initialized. Note that workers > 1 cannot be specified when not in main.")
# ================================================================
data = [] # list of dicts to be filled with dicts...

with torch.no_grad():
    for images, label, paths in tqdm(dataloader, total=len(dataloader)):
        # Send stuff to GPU if available.
        images = images.to(device)
        label = label.to(device)
        # Make Forward Pass.
        outputs = model(images)
        _, preds = torch.max(outputs, dim=1)
        # path_list = []
        # for path in paths:
        #     path_list.append(path)
        
        for i, image in enumerate(images):
            private_dict = {}
            # Three entries regarding the image identification.
            # private_dict['path'] = path_list[i]
            private_dict['path'] = paths[i]
            private_dict['layer'] = label[i].item()
            private_dict['prediction'] = preds[i].item()
            # Iterate over all available layers.
            for key, hook_object in layer_activations.items():
                tensor_out = hook_object()  # .output  # modified from original script to accommodate objects 
                if key == 'fc':
                    # The array to store is a 32 by 10 array, each batch
                    output = torch.unbind(tensor_out, dim=0)
                else:
                    # The array will have a final shape of 32 by num_channels
                    # in specific layer
                    b, c, _, _ = tensor_out.shape
                    output = torch.unbind(tensor_out.view(b, c, -1).mean(2), dim=0)
                private_dict[key] = output[i].cpu().numpy()
            data.append(private_dict)

Dataloader Initialized. Note that workers > 1 cannot be specified when not in main.


100%|██████████| 262/262 [06:44<00:00,  1.54s/it]


In [6]:
df = pd.DataFrame(data, copy=False)
# df.head()

In [7]:
df.to_parquet(r"C:\Users\Noel\Documents\THESIS\Feature Visualization\Dataframes\reapeatability_resnet34.parquet")

In [9]:
df = pd.read_parquet(r"C:\Users\Noel\Documents\THESIS\Feature Visualization\Dataframes\reapeatability_resnet34.parquet")
df.tail()

,path,layer,prediction,conv1,layer1.0.conv1,layer1.0.conv2,layer1.1.conv1,layer1.1.conv2,layer1.2.conv1,layer1.2.conv2,...,layer3.5.conv1,layer3.5.conv2,layer4.0.conv1,layer4.0.conv2,layer4.0.downsample.0,layer4.1.conv1,layer4.1.conv2,layer4.2.conv1,layer4.2.conv2,fc
16763,C:\Users\Noel\Documents\THESIS\Outputs_Feature...,35,1,"[0.00096876634, 0.114484824, -0.04415039, -0.0...","[-0.057529658, -0.063573174, 0.038570747, 0.03...","[-0.008665042, -0.0071253264, -0.0025478743, -...","[0.0007617282, -0.015230497, -0.003674686, -0....","[-0.0032790632, -4.3875396e-05, -0.0003813878,...","[-0.012611017, -0.0005725565, -0.0104052825, -...","[-0.0039874683, -0.0008782879, -0.0023062448, ...",...,"[-0.0033337115, -0.0027309558, -0.006235859, -...","[0.0005360425, -9.052953e-05, -0.0021928581, -...","[0.0026599306, 0.025986698, 0.010221425, -0.00...","[-0.0027279442, 0.0022271308, -2.4134255e-05, ...","[-0.00267472, -0.0027023633, -0.0010572865, -0...","[0.003845722, 0.0023889209, -0.00022369532, -0...","[-0.005443983, 0.0054930556, 0.0013585001, -0....","[-0.030245243, -0.014614122, 0.00019472624, -0...","[-0.007139252, 0.008308258, 0.00093443657, 9.3...","[-2.0899808, 7.1576595, -1.6357356, 0.185611, ..."
16764,C:\Users\Noel\Documents\THESIS\Outputs_Feature...,35,2,"[0.0011316112, 0.103750035, -0.026638087, -0.0...","[-0.059302013, -0.06850745, 0.04201432, 0.0321...","[-0.008857502, -0.007807972, -0.002084257, -0....","[0.00080051523, -0.01521659, -0.0036875526, -0...","[-0.0035028323, -0.00029173162, -0.00086190103...","[-0.013251992, -0.0015560512, -0.009748482, -0...","[-0.004242531, -0.00091371575, -0.0022335786, ...",...,"[-0.012142185, -0.010446279, -0.02594975, -0.0...","[-0.0011639284, 0.0006540309, -0.004402735, -0...","[0.017940098, 0.058991026, 0.032060556, -0.000...","[0.0048445384, -0.024306469, -0.002932472, 0.0...","[-0.010082682, -0.011240369, -0.0037485347, 0....","[0.11149411, 0.079004765, -0.09613093, 0.10035...","[-0.024657998, -0.027079722, -0.011865776, 0.0...","[0.0042196717, 0.094611324, 0.12524557, 0.0765...","[-0.036866132, -0.019842548, -0.018178288, 0.0...","[-0.6999701, -1.7772549, 6.829746, -4.4503675,..."
16765,C:\Users\Noel\Documents\THESIS\Outputs_Feature...,35,3,"[-0.0001686031, -0.044705294, 0.038685407, 0.0...","[-0.050312568, -0.0669264, 0.039513994, 0.0240...","[-0.007111615, -0.0045274147, -0.0019259733, -...","[0.0012283836, -0.014501246, -0.0038110767, -0...","[-0.0034099142, -0.0003029082, -0.0007221217, ...","[-0.010378127, 0.00044186704, -0.009007157, -0...","[-0.006328371, -0.0012991672, -0.0021493915, -...",...,"[0.0057261027, -0.029653726, 0.013501534, -0.0...","[0.00396108, 0.009770911, -0.025450194, -0.032...","[-0.02280678, -0.051518936, -0.043059252, -0.0...","[0.061880767, 0.15515295, 0.033979557, -0.0028...","[0.020194318, 0.06499129, 0.010237767, -0.0079...","[-0.10511552, -0.062572956, -0.12431853, -0.35...","[0.044749025, 0.11800861, 0.022330768, 0.00354...","[-0.25621364, -0.40510118, -0.20398428, -0.126...","[0.025166823, 0.06821161, 0.009425057, 0.00052...","[-2.2767293, -3.6783912, -1.3631047, 16.932001..."
16766,C:\Users\Noel\Documents\THESIS\Outputs_Feature...,35,2,"[-1.0849307e-05, -0.018928057, 0.015330843, 0....","[-0.04425906, -0.05638759, 0.031417742, 0.0236...","[-0.0051782485, -0.0027342706, -0.0016419498, ...","[0.0020780987, -0.013625809, -0.0018170886, -0...","[-0.0027625882, -0.00021597916, -0.0002880661,...","[-0.00941255, 0.0006703108, -0.009750235, -0.0...","[-0.0043701907, -0.00060331787, -0.001960252, ...",...,"[-0.02635998, -0.030668037, -0.04729579, -0.01...","[-0.004548288, 0.006140421, -0.022189068, -0.0...","[-0.017954288, -0.023509398, -0.02389037, -0.0...","[-0.01211678, -0.014070711, -0.0033538754, 0.0...","[-0.009657426, -0.0068088933, -0.0010294435, 0...","[0.25229046, 0.04529514, -0.0115074515, 0.0872...","[-0.0074387137, -0.0118736075, -0.0059353607, ...","[-0.11042725, 0.0126648825, 0.18875137, -0.061...","[-0.007405166, -0.011222391, -0.0070979